In [12]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [13]:
def print_matrix(M: np.ndarray):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    """
    for row in M:
        print(f"{np.array2string(row, precision=2 ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    """
    
    eq_stat = (np.angle(V1) == np.angle(V2))
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [14]:
U = itf.random_unitary(5)

# Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [15]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 -0.38-0.28j, -0.45-0.33j
Corresponding angles:	 -2.51, -2.51

They are NOT matching!

Full length of angles:
 -2.508033813367406, -2.5080338133674056

Their difference:	 -4.440892098500626e-16


## Finding internal phaseshifts
$\delta$ and $\sum$

In [16]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M2:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.87-0.00j	 -> real: False
angle:	-180.0°

Block of M2:

[-0.76-1.02e-16j,  0.64-1.22e-16j]
[0.64-1.22e-16j, 0.76+1.02e-16j]




In [17]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 7.63°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [18]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, j, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.41+0.34j, -0.16+0.14j
Corresponding angles:	 2.45, 2.45

They are matching!


## Finding internal phaseshifts
$\delta$ and $\sum$

In [19]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta))

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M2:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# # print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.38-0.00j	 -> real: True
angle:	-0.0°


In [20]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 7.70°

Elements affected by e^(i*summ):

Affected elements:	 -0.58-0.43j, -0.08-0.06j
Corresponding angles:	 -2.51, -2.51

They are NOT matching!

Full length of angles:
 -2.508033813367406, -2.5080338133674065

Their difference:	 4.440892098500626e-16


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [21]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)
V = np.matmul(M,V)

# print('\nBlock of M2:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.14-0.00j	 -> real: False
angle:	-3.607799565659417e-14°


In [22]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.58) Im(-0.43)
abs: 0.73, angle: -143.70°

